# Type-A: TF-IDF & LDA 
With this version, we are going to perform TF-IDF then LDA.  
We can then find similar documents from the corpus.  

## Libraries

In [340]:
# Imports 
import json 
import csv
import  itertools
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from smart_open import open
from bs4 import BeautifulSoup
from gensim.models import TfidfModel

## Pre processing

In [2]:
import pandas as pd

In [10]:
df = pd.read_json('./data/how-good-is-your-medium-article/train.json', lines=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62313 entries, 0 to 62312
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _id         62313 non-null  object 
 1   _timestamp  62313 non-null  float64
 2   _spider     62313 non-null  object 
 3   url         62313 non-null  object 
 4   domain      62313 non-null  object 
 5   published   62313 non-null  object 
 6   title       62313 non-null  object 
 7   content     62313 non-null  object 
 8   author      62313 non-null  object 
 9   image_url   57226 non-null  object 
 10  tags        62313 non-null  object 
 11  link_tags   62313 non-null  object 
 12  meta_tags   62313 non-null  object 
dtypes: float64(1), object(12)
memory usage: 6.2+ MB


In [40]:
columns2keep = ['url', 'title', 'author', 'image_url']

In [13]:
df[columns2keep].head()

,url,title,content,author,image_url
0,https://medium.com/policy/medium-terms-of-serv...,Medium Terms of Service – Medium Policy – Medium,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@Med...",None
1,https://medium.com/policy/amendment-to-medium-...,Amendment to Medium Terms of Service Applicabl...,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@Med...",None
2,https://medium.com/@aelcenganda/%E9%96%A9%E6%9...,走入山與海之間：閩東大刀會和兩岸走私 – Yun-Chen Chien（簡韻真） – Medium,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@ael...",https://cdn-images-1.medium.com/max/1200/1*k9f...
3,https://medium.com/what-comes-to-mind/how-fast...,How fast can a camera get? – What comes to min...,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@vai...",https://cdn-images-1.medium.com/max/1200/1*6UZ...
4,https://medium.com/what-comes-to-mind/a-game-f...,A game for the lonely fox – What comes to mind...,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@vai...",https://cdn-images-1.medium.com/max/1200/1*liU...


In [14]:
df['content'][0]

'<div><header class="container u-maxWidth740"><div class="uiScale uiScale-ui--regular uiScale-caption--regular postMetaHeader u-paddingBottom10 row"><div class="col u-size12of12 js-postMetaLockup"><div class="uiScale uiScale-ui--regular uiScale-caption--regular postMetaLockup postMetaLockup--authorWithBio u-flexCenter js-postMetaLockup"><div class="u-flex0"><a class="link u-baseColor--link avatar" href="https://medium.com/@Medium?source=post_header_lockup" data-action="show-user-card" data-action-source="post_header_lockup" data-action-value="504c7870fdb6" data-action-type="hover" data-user-id="504c7870fdb6" dir="auto"><div class="u-relative u-inlineBlock u-flex0"><img src="https://cdn-images-1.medium.com/fit/c/120/120/1*6_fgYnisCa9V21mymySIvA.png" class="avatar-image avatar-image--small" alt="Go to the profile of Medium"><div class="avatar-halo u-absolute u-textColorGreenNormal svgIcon" style="width: calc(100% + 12px); height: calc(100% + 12px); top:-6px; left:-6px"><svg viewbox="0 0 

### Extract the content from the html tags

In [21]:
soup = BeautifulSoup(df['content'][0], 'html.parser')
data_block = soup.getText()
data_block

'MediumEveryone’s stories and ideasAug 13, 2012Medium Terms of\xa0ServiceEffective: March 7, 2016These Terms of Service (“Terms”) are a contract between you and A Medium Corporation. They govern your use of Medium’s sites, services, mobile apps, products, and content (“Services”).By using Medium, you agree to these Terms. If you don’t agree to any of the Terms, you can’t use Medium.We can change these Terms at any time. We keep a historical record of all changes to our Terms on GitHub. If a change is material, we’ll let you know before they take effect. By using Medium on or after that effective date, you agree to the new Terms. If you don’t agree to them, you should delete your account before they take effect, otherwise your use of the site and content will be subject to the new Terms.Content rights & responsibilitiesYou own the rights to the content you create and post on Medium.By posting content to Medium, you give us a nonexclusive license to publish it on Medium Services, includi

In [27]:
def extract_html_content(row, column_name='content'): 
    """
    Takes in a Row from a pandas DF. 
    Returns the text from the specified html in the given column-name
    
    Example: extract_html_content(df.iloc[0], 'content')
    """
    soup = BeautifulSoup(row[column_name], 'html.parser')
    return soup.getText()

In [28]:
df['html_text'] = df.apply(extract_html_content, axis='columns')

In [29]:
df.head()

,_id,_timestamp,_spider,url,domain,published,title,content,author,image_url,tags,link_tags,meta_tags,html_text
0,https://medium.com/policy/medium-terms-of-serv...,1.520035e+09,medium,https://medium.com/policy/medium-terms-of-serv...,medium.com,{'$date': '2012-08-13T22:54:53.510Z'},Medium Terms of Service – Medium Policy – Medium,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@Med...",None,[],{'canonical': 'https://medium.com/policy/mediu...,"{'viewport': 'width=device-width, initial-scal...","MediumEveryone’s stories and ideasAug 13, 2012..."
1,https://medium.com/policy/amendment-to-medium-...,1.519336e+09,medium,https://medium.com/policy/amendment-to-medium-...,medium.com,{'$date': '2015-08-03T07:44:50.331Z'},Amendment to Medium Terms of Service Applicabl...,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@Med...",None,[],{'canonical': 'https://medium.com/policy/amend...,"{'viewport': 'width=device-width, initial-scal...","MediumEveryone’s stories and ideasAug 2, 2015 ..."
2,https://medium.com/@aelcenganda/%E9%96%A9%E6%9...,1.519336e+09,medium,https://medium.com/@aelcenganda/%E9%96%A9%E6%9...,medium.com,{'$date': '2017-02-05T13:08:17.410Z'},走入山與海之間：閩東大刀會和兩岸走私 – Yun-Chen Chien（簡韻真） – Medium,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@ael...",https://cdn-images-1.medium.com/max/1200/1*k9f...,[],{'canonical': 'https://medium.com/@aelcenganda...,"{'viewport': 'width=device-width, initial-scal...","Yun-Chen Chien（簡韻真）Nobody in @g0v.tw, PM in se..."
3,https://medium.com/what-comes-to-mind/how-fast...,1.519337e+09,medium,https://medium.com/what-comes-to-mind/how-fast...,medium.com,{'$date': '2017-05-06T08:16:30.776Z'},How fast can a camera get? – What comes to min...,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@vai...",https://cdn-images-1.medium.com/max/1200/1*6UZ...,[],{'canonical': 'https://medium.com/what-comes-t...,"{'viewport': 'width=device-width, initial-scal...",Vaibhav KhulbeAndroid App Developer | I write ...
4,https://medium.com/what-comes-to-mind/a-game-f...,1.519337e+09,medium,https://medium.com/what-comes-to-mind/a-game-f...,medium.com,{'$date': '2017-06-04T14:46:25.772Z'},A game for the lonely fox – What comes to mind...,"<div><header class=""container u-maxWidth740""><...","{'name': None, 'url': 'https://medium.com/@vai...",https://cdn-images-1.medium.com/max/1200/1*liU...,[],{'canonical': 'https://medium.com/what-comes-t...,"{'viewport': 'width=device-width, initial-scal...",Vaibhav KhulbeAndroid App Developer | I write ...


### Save samples with the needed columns as CSVs 
Samples to save:  
* 500 samples  
* 1000 samples   
* 10000 samples  
* 30000 samples  
* Whole document  

In [41]:
columns2keep.append('html_text')
columns2keep

['url', 'title', 'author', 'image_url', 'html_text']

In [46]:
df[columns2keep].sample(2).to_csv('sample.csv')

In [47]:
counts = [500, 1000, 5000, 10000, 25000]
for count in counts: 
    print(f'Starting {count} count saving...')
    df[columns2keep].sample(count).to_csv(f'./data/samples/sample{count}.csv')

Starting 500 count saving...
Starting 1000 count saving...
Starting 5000 count saving...
Starting 10000 count saving...
Starting 25000 count saving...


In [48]:
# Save the whole document.
df[columns2keep].sample(frac=1).to_csv(f'./data/samples/sampleFull.csv')

## Training Models

### Prep the Data 

In [351]:
# Load in Dataset 
class MyCorpusCSV: 
    def __init__(self, csv_link, idx): 
        # idx is the index of the row where the text content is
        self.csv_link = csv_link 
        self.content_idx = idx
        self.count = 0
    
    def __len__(self): 
        return self.count
    
    def get_nth(self, n): 
        return next(itertools.islice(self.generator(), n, None))
    
    def generator(self): 
        with open(self.csv_link) as csv_file: 
            reader = csv.reader(csv_file)
            for row in reader:
                yield row
        
    def __iter__(self):  
        with open(self.csv_link) as csv_file: 
            reader = csv.reader(csv_file)
            next(reader, None)
            for row in reader:
                self.count += 1
                yield row[self.content_idx]

#                 yield row[self.content_idx] # To get the whole doc, use this instead

In [352]:
link = './data/samples/sample500.csv'
# link = './data/samples/test.csv'
content_col_number = 5
mycorpus = MyCorpusCSV(link, content_col_number)

In [284]:
# Create a dictionary of all words in corpus - remove the stopwords in the process
dictionary = corpora.Dictionary()
# Add document to dictionary 
for doc in mycorpus: 
    dictionary.add_documents([doc.lower().split()])

In [285]:
print(dictionary)

Dictionary(120975 unique tokens: ['$1.3m', '$100', '$13', '$14', '(“nellie']...)


In [286]:
# Remove stopwords 
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in STOPWORDS
    if stopword in dictionary.token2id
]
# Remove id's that occur once 
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(once_ids + stop_ids)
dictionary.compactify()

In [287]:
print(dictionary)

Dictionary(27819 unique tokens: ['$100', '11,', '2014,', '2017what', 'ability']...)


## Testing Models

In [313]:
# Load in Dataset 
class BoWCorpusCSV: 
    def __init__(self, csv_link, idx): 
        # idx is the index of the row where the text content is
        self.csv_link = csv_link 
        self.content_idx = idx
        self.count = 0
    
    def __len__(self): 
        return self.count
        
    def __iter__(self):  
        from gensim.parsing.preprocessing import remove_stopwords
        with open(self.csv_link) as csv_file: 
            reader = csv.reader(csv_file)
            next(reader, None)
            for row in reader:
                self.count += 1
                yield dictionary.doc2bow(row[self.content_idx].lower().split())


## TF-IDF Models

In [314]:
bowcorpus = BoWCorpusCSV(link, content_col_number)

In [318]:
tfidf_model = TfidfModel(bowcorpus)

In [316]:
# len(next(bowcorpus.__iter__()))

372

### Convert a sample text to TFIDF document

In [321]:
text = 'Using machine learning, one can find useful patterns from large data sets to make'\
'data more informative and qualitatively insightful. This is very important for'\
'decision making. Students will be exposed to supervised and unsupervised'\
'learning, respectively.'

In [324]:
text_words = simple_preprocess(text)
tfidf_vec = tfidf_model[dictionary.doc2bow(text_words)]
print(tfidf_vec)

[(153, 0.10382537902495723), (1195, 0.1061252121871431), (1948, 0.1517823003148463), (2039, 0.29525705725800117), (2283, 0.1628713117927232), (2290, 0.17424377512633324), (2429, 0.20225556912127385), (3320, 0.32659558027188684), (4144, 0.24001998543500375), (6386, 0.24250382625146066), (12036, 0.476496230514886), (12077, 0.3045786815346222), (17922, 0.476496230514886)]


## Get similar documents from the corpus

In [330]:
index = similarities.Similarity(None, bowcorpus, len(dictionary))

In [332]:
res = index[tfidf_vec]

In [334]:
# Print top 10 documents 
print(list(sorted(enumerate(res), key=lambda x: x[1], reverse=True))[:10])

[(43, 0.11912215), (281, 0.10735533), (396, 0.1049601), (94, 0.100949824), (125, 0.091222815), (389, 0.0863544), (291, 0.08232404), (97, 0.08066791), (243, 0.0803193), (118, 0.080146275)]


In [360]:
# Get the nth value from the corpus to confirm 
import itertools
index = 281
mycorpus.get_nth(index)
# next(itertools.islice(mycorpus.get_nth(), index, None))

['49816',
 'https://medium.com/@clintperez/affecting-change-how-modern-marketing-tools-build-ongoing-relationships-in-advocacy-b5c1b9f53a21',
 'Affecting Change: How Modern Marketing Tools Build Ongoing Relationships in Advocacy',
 "{'name': None, 'url': 'https://medium.com/@clintperez', 'twitter': '@clintperez'}",
 'https://cdn-images-1.medium.com/max/1200/1*eI49N0CWPsKr_BfBUglQxg.jpeg',
 'Clint PerezAvid homebrewer, distance runner,  sports enthusiast and tech geek. Dedicated husband and father. Native Washingtonian. Tweets are my own.Nov 4, 2016Affecting Change: How Modern Marketing Tools Build Ongoing Relationships in\xa0AdvocacyQ&A with Bryan Fratkin, Founder & CEO, Whistle Stop\xa0DigitalBryan Fratkin is a native Washingtonian. He has spent his entire career at the nexus of politics, advocacy and digital marketing. But those worlds never came together quite the way he wanted them to until he founded Whistle Stop Digital, makers of SparkInfluence. “I was tired of stitching togethe

## Saving Models